<a href="https://colab.research.google.com/github/kkashleva/news-scraping-competition/blob/main/%22Baseline_NLP2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline-решение

По мотивам ноутбука https://www.kaggle.com/code/hardtype/parsing-news-from-rbc-lenta-ru

## 1. Парсим новости с сайта Lenta.ru

In [ ]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=bd63765260431c89cf7d8c6289222c44266bf1258b5a02abe4e622315259244d
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 20,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [ ]:
# Задаем тут параметры
query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0"
dateFrom = '2018-01-01' # увеличиваем объем
dateTo = "2023-12-30"

param_dict = {'query'     : query,
              'from'      : str(offset),
              'size'      : str(size),
              'dateFrom'  : dateFrom,
              'dateTo'    : dateTo,
              'sort'      : sort,
              'title_only': title_only,
              'type'      : material,
              'bloc'      : bloc,
              'domain'    : domain}

print("param_dict:", param_dict)

param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2018-01-01', 'dateTo': '2023-12-30', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [ ]:
parser = lentaRu_parser()

tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 20, # уменьшаем шаг
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
105000


,rightcol,docid,image_url,lastmodtime,part,title,type,url,tags,bloc,domain,modified,text,status,pubdate,snippet
0,Чрезвычайных ситуаций во время празднования Но...,814958,https://icdn.lenta.ru/images/2018/01/01/03/201...,1514766248,0,МЧС отчиталось о первых часах наступившего года,1,https://lenta.ru/news/2018/01/01/mchs/,[2],1,1,1514766180,Фото: Максим Блинов / РИА Новости Чрезвычайных...,0,1514766180,Фото: Максим Блинов / РИА Новости ... ситуация...
1,Препарат для борьбы с диабетом помогает против...,814887,,1514768455,0,Обнаружено неожиданное средство от старения мозга,1,https://lenta.ru/news/2018/01/01/diabet/,[16],5,1,1514768400,Ученые Ланкастерского университета в Великобри...,0,1514768400,"Ученые Ланкастерского университета в ..., в то..."
2,Президент США раскритиковал решение иранских в...,814959,https://icdn.lenta.ru/images/2018/01/01/04/201...,1514770871,0,Трамп оценил блокировку Ираном Telegram и Inst...,1,https://lenta.ru/news/2018/01/01/trump/,[31],7,1,1514769660,Дональд Трамп Фото: Shannon Stapleton / Reuter...,0,1514769660,Дональд Трамп Фото: Shannon Stapleton / ... к ...
3,"«Ядерная кнопка лежит на моем рабочем столе», ...",814960,https://icdn.lenta.ru/images/2018/01/01/04/201...,1514801286,0,Северная Корея завершила создание национальных...,1,https://lenta.ru/news/2018/01/01/gotovo/,[1],2,1,1514771880,Ким Чен Ын Фото: ЦТАК / Reuters Северная Корея...,0,1514771880,Ким Чен Ын Фото: ЦТАК / Reuters Северная ... з...
4,С 1 января 2018 года МРОТ устанавливается на у...,814961,https://icdn.lenta.ru/images/2018/01/01/05/201...,1514775274,0,Минимальный размер оплаты труда россиян увелич...,1,https://lenta.ru/news/2018/01/01/mrot/,[7],4,1,1514775240,Фото: Александр Кряжев / РИА Новости В России ...,0,1514775240,Фото: Александр Кряжев / РИА Новости В ... от ...


In [ ]:
tbl.to_csv("Lenta_sample.csv", index=False)

In [ ]:
tbl = pd.read_csv("Lenta_sample.csv")

In [ ]:
tbl.shape

(105000, 16)

Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - Россия - 0
* 37 - Силовые структуры - 2
* 3 - Бывший СССР - 3
* 4 - Экономика - 1
* 5 - Наука и техника - 8
* 8 - Спорт - 4
* 48 - Туризм - 7
* 87 - Здоровье - 5

In [ ]:
tbl[tbl.bloc == 5].iloc[0]

rightcol       Препарат для борьбы с диабетом помогает против...
docid                                                     814887
image_url                                                    NaN
lastmodtime                                           1514768455
part                                                           0
title          Обнаружено неожиданное средство от старения мозга
type                                                           1
url                     https://lenta.ru/news/2018/01/01/diabet/
tags                                                        [16]
bloc                                                           5
domain                                                         1
modified                                              1514768400
text           Ученые Ланкастерского университета в Великобри...
status                                                         0
pubdate                                               1514768400
snippet        Ученые Лан

In [ ]:
tbl['bloc'].value_counts(normalize=True)

1     0.175419
2     0.132610
4     0.100200
3     0.089362
8     0.072276
37    0.060905
7     0.054781
5     0.053981
9     0.047267
6     0.047133
12    0.042029
48    0.039457
47    0.039295
86    0.010914
53    0.010095
87    0.009790
0     0.007486
49    0.006343
52    0.000400
51    0.000152
40    0.000067
11    0.000029
35    0.000010
Name: bloc, dtype: float64

In [ ]:
tbl = tbl[tbl.bloc.isin([1, 37, 3, 4, 5, 8, 48, 87])] #оставляем только нужные темы

TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5} #соотносим с темами в соревновании

tbl['topic'] = tbl['bloc'].map(TagsMap)

<ipython-input-11-961ce383769a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl['topic'] = tbl['bloc'].map(TagsMap)


In [ ]:
tbl.shape

(63146, 17)

In [ ]:
tbl['topic'].value_counts(normalize=True) # можно сверить с распределением меток классов в соревновании

0    0.291689
1    0.166614
3    0.148592
4    0.120182
2    0.101273
8    0.089760
7    0.065610
5    0.016280
Name: topic, dtype: float64

In [ ]:
tbl.head()

,rightcol,docid,image_url,lastmodtime,part,title,type,url,tags,bloc,domain,modified,text,status,pubdate,snippet,topic
0,Чрезвычайных ситуаций во время празднования Но...,814958,https://icdn.lenta.ru/images/2018/01/01/03/201...,1514766248,0,МЧС отчиталось о первых часах наступившего года,1,https://lenta.ru/news/2018/01/01/mchs/,[2],1,1,1514766180,Фото: Максим Блинов / РИА Новости Чрезвычайных...,0,1514766180,Фото: Максим Блинов / РИА Новости ... ситуация...,0
1,Препарат для борьбы с диабетом помогает против...,814887,NaN,1514768455,0,Обнаружено неожиданное средство от старения мозга,1,https://lenta.ru/news/2018/01/01/diabet/,[16],5,1,1514768400,Ученые Ланкастерского университета в Великобри...,0,1514768400,"Ученые Ланкастерского университета в ..., в то...",8
4,С 1 января 2018 года МРОТ устанавливается на у...,814961,https://icdn.lenta.ru/images/2018/01/01/05/201...,1514775274,0,Минимальный размер оплаты труда россиян увелич...,1,https://lenta.ru/news/2018/01/01/mrot/,[7],4,1,1514775240,Фото: Александр Кряжев / РИА Новости В России ...,0,1514775240,Фото: Александр Кряжев / РИА Новости В ... от ...,1
5,На месте сгоревшего дерева установят другую ель,814962,https://icdn.lenta.ru/images/2018/01/01/06/201...,1514777872,0,Названа возможная причина уничтожившего елку в...,1,https://lenta.ru/news/2018/01/01/prichina/,[2],1,1,1514777760,Фото: пресс-служба мэрии Южно-Сахалинска Пожар...,0,1514777760,"Фото: пресс-служба мэрии Южно-Сахалинска ..., ...",0
7,Глава Чечни пожелал всем в наступившем году сч...,814964,https://icdn.lenta.ru/images/2018/01/01/07/201...,1514801451,7,Кадыров прочитал новогоднее стихотворение,1,https://lenta.ru/news/2018/01/01/stih/,[1],1,1,1514782560,Рамзан Кадыров Фото: Илья Питалев / РИА Новост...,0,1514782560,Рамзан Кадыров Фото: Илья Питалев / РИА ... и ...,0


In [ ]:
tbl_final = tbl.drop(columns=['docid', 'url', 'title', 'modified', 'lastmodtime', 'type', 'domain', 'status', 'part', 'image_url', 'pubdate', 'rightcol', 'snippet', 'tags', 'bloc'])

In [ ]:
tbl_final.head()

,text,topic
0,Фото: Максим Блинов / РИА Новости Чрезвычайных...,0
1,Ученые Ланкастерского университета в Великобри...,8
4,Фото: Александр Кряжев / РИА Новости В России ...,1
5,Фото: пресс-служба мэрии Южно-Сахалинска Пожар...,0
7,Рамзан Кадыров Фото: Илья Питалев / РИА Новост...,0


In [ ]:
df_final = pd.read_csv('FULL_data_tokenized_v3.csv') # добавляем небольшой готовый датасет отсюда https://www.kaggle.com/code/godblessroman/rubert-fine-tuning-multiclf

In [ ]:
df_final.head()

,0,target,processed
0,Массовые задержания мигрантов произошли в ново...,0,массовый задержание мигрант происходить нового...
1,С 29 декабря открыты поставки инкубационного я...,0,декабрь открывать поставка инкубационный яйцо ...
2,"На востоке Ленобласти с 31 декабря, с 1 января...",0,восток ленобласть декабрь январь весь территор...
3,"Петербургский бизнесмен, основатель ЧВК «Вагне...",0,петербургский бизнесмен основатель чвк вагнер ...
4,Спецборт МЧС России вылетел из Каира в Москву ...,0,спецборт мчс россия вылетать каир москва эваку...


In [ ]:
df_final = df_final.rename(columns={'0': 'text', 'target': 'topic'})

In [ ]:
df1 = df_final.drop(columns = ['processed'])
df1.head()

,text,topic
0,Массовые задержания мигрантов произошли в ново...,0
1,С 29 декабря открыты поставки инкубационного я...,0
2,"На востоке Ленобласти с 31 декабря, с 1 января...",0
3,"Петербургский бизнесмен, основатель ЧВК «Вагне...",0
4,Спецборт МЧС России вылетел из Каира в Москву ...,0


In [ ]:
final = pd.concat([tbl_final, df1])


In [ ]:
final.to_csv('lenta_parsed_lentaf.csv',index=False )

In [ ]:
final.head()

,text,topic
0,Фото: Максим Блинов / РИА Новости Чрезвычайных...,0
1,Ученые Ланкастерского университета в Великобри...,8
4,Фото: Александр Кряжев / РИА Новости В России ...,1
5,Фото: пресс-служба мэрии Южно-Сахалинска Пожар...,0
7,Рамзан Кадыров Фото: Илья Питалев / РИА Новост...,0


## 2. Машинное обучение

Загружаем данные и обучаем модель на разбиении трейн-тест

In [ ]:
final = pd.read_csv('lenta_parsed_lentaf.csv')
final_new = final[~final.text.isna()] # убираем пустые новости

print(len(final), len(final_new))

64153 63177


In [ ]:
final_new.head()

,text,topic
0,Фото: Максим Блинов / РИА Новости Чрезвычайных...,0
1,Ученые Ланкастерского университета в Великобри...,8
2,Фото: Александр Кряжев / РИА Новости В России ...,1
3,Фото: пресс-служба мэрии Южно-Сахалинска Пожар...,0
4,Рамзан Кадыров Фото: Илья Питалев / РИА Новост...,0


In [ ]:
X = final_new[['text']]
y = final_new['topic']

X.shape

(63177, 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=final_new.topic)

In [ ]:
X_train.shape, X_test.shape

((47382, 1), (15795, 1))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

vec = TfidfVectorizer(analyzer='word') # выбираем TfidfVectorizer
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler() # масштабирование данных (можно использовать, если нет отрицательных значений):
#для линейных моделей градиентный спуск работает быстрее, при регуляризации. особенно важно для _ЛогРег_ (для бустингов и лесов неважно)
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(random_state=0)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(47382, 301041)
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      4619
           1       0.92      0.92      0.92      2648
           2       0.92      0.89      0.91      1618
           3       0.93      0.93      0.93      2362
           4       0.98      0.98      0.98      1721
           5       0.94      0.91      0.92       280
           6       0.92      0.52      0.67        23
           7       0.96      0.90      0.93      1075
           8       0.95      0.91      0.93      1449

    accuracy                           0.92     15795
   macro avg       0.93      0.87      0.90     15795
weighted avg       0.92      0.92      0.92     15795



Загружаем тестовые данные, обучаем итоговую модель и делаем прогноз.

In [ ]:
Test = pd.read_csv("test_news.csv")
Test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report

vec = TfidfVectorizer(analyzer='word')
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(random_state=0)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [ ]:
pred[:10], len(pred)

(array([0, 6, 4, 0, 4, 3, 2, 3, 3, 3]), 26275)

Сохраняем прогноз в файл.

In [ ]:
subm = pd.read_csv("base_submission_news.csv")
subm.head()

,topic,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [ ]:
subm['topic'] = pred

subm.to_csv("my_baseline_submission.csv", index=False)

In [ ]:
subm['topic'].value_counts(normalize=True)

0    0.368335
1    0.142569
3    0.128411
4    0.096746
2    0.090466
8    0.070942
7    0.055680
5    0.023520
6    0.023330
Name: topic, dtype: float64

при дисбалансе классов:

использовать вероятностные модели

class_weight = balanced
